In [27]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import pandas as pd 
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

In [28]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
params = {'page': 1}
url = "https://hh.ru"
session = requests.Session()

In [30]:
world = input('Введите значение слова поиска: ')
#world = 'Python'

Введите значение слова поиска: Python


In [31]:
world

'Python'

In [32]:
n=1

In [33]:
# Подключаем базу
client = MongoClient('127.0.0.1', 27017)

In [35]:
response = session.get('https://hh.ru/search/vacancy?text=' + world + '&from=suggest_post', headers=headers,params=params)

In [36]:
response

<Response [200]>

In [38]:
db = client['vacancy']


In [39]:
persons= db.persons # collection

In [56]:
while response and n<3:
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
    for article in articles:
        info = article.find('a', {'class': 'serp-item__title'})
        vacname=info.text
        link = article.find('a').get('href')
        salary = article.find('span', {'class': 'bloko-header-section-2'}) 
        is_check_sarary =''
        salary_currency =''
        salary_max =''
        salary_min =''
        vtmp1 =''
        vtmp2 =''
        v= ''
        if not salary:
            pass
        else:
            is_check_sarary = salary.text.replace(u'\u202f', u' ')
            v = is_check_sarary[is_check_sarary.rfind(' ')+1:]
            is_check_sarary =is_check_sarary.replace(v, '')
            is_check_sarary =is_check_sarary.replace(' ', '')
            k =0
            for i in is_check_sarary:
                #i.isdigit()
                if i.isdigit() and k==0:
                    vtmp1=vtmp1+i
                elif i == ' ' and k==0:
                    pass
                elif vtmp1 !='':
                    k=1
                else:
                    pass
            k=0
            for i in reversed(is_check_sarary): 
                if i.isdigit() and k==0:
                    vtmp2=i+vtmp2
                elif i == ' ' and k==0:
                    pass
                elif vtmp2 !='':
                    k=1
                else:
                    pass
            if 'до' in is_check_sarary:
                salary_min =''
                salary_max = vtmp2
            elif 'от' in is_check_sarary:
                salary_min = vtmp1
                salary_max = ''
            elif '–' in is_check_sarary:
                salary_min = vtmp1
                salary_max = vtmp2
            else:
                salary_currency = vtmp1
        #Уникальность вакансии проверяем по ссылке, ее и подставляем в id
        if persons.find_one({'_id': link}):
            pass
        else:
            persons.insert_one({'vacancy': vacname, '_id': link, 'salary_min': salary_min, 'salary_currency': salary_currency, 'salary_max': salary_max, 'v': v,'key':world})
            #print(vacname);        
    response = session.get('https://hh.ru/search/vacancy?text=' + world + '&from=suggest_post', headers=headers,params=params)
    n=n+1


In [59]:
for doc in persons.find({}):
    pprint(doc)

{'_id': 'https://hh.ru/vacancy/84418444?from=vacancy_search_list&query=Python',
 'key': 'Python',
 'salary_currency': '',
 'salary_max': '',
 'salary_min': '',
 'v': '',
 'vacancy': 'Python разработчик'}
{'_id': 'https://hh.ru/vacancy/84255351?from=vacancy_search_list&query=Python',
 'key': 'Python',
 'salary_currency': '',
 'salary_max': '380000',
 'salary_min': '250000',
 'v': '₽',
 'vacancy': 'Data Scientist middle+/Senior'}
{'_id': 'https://hh.ru/vacancy/84260215?from=vacancy_search_list&query=Python',
 'key': 'Python',
 'salary_currency': '',
 'salary_max': '170000',
 'salary_min': '150000',
 'v': '₽',
 'vacancy': 'ГИС-разработчик (Python)'}
{'_id': 'https://hh.ru/vacancy/84001239?from=vacancy_search_list&query=Python',
 'key': 'Python',
 'salary_currency': '',
 'salary_max': '300000',
 'salary_min': '',
 'v': '₽',
 'vacancy': 'Программист Python'}
{'_id': 'https://hh.ru/vacancy/83655667?from=vacancy_search_list&query=Python',
 'key': 'Python',
 'salary_currency': '',
 'salary_max

In [117]:
def vacmin(salarymin):
    for doc in persons.find({}):
    #if doc[salary_max]=='':
        if doc.get('salary_max')!='':
            if doc.get('salary_min')!='':
                sm = doc.get('salary_min')
                smd = int(sm)
                if smd>salarymin:
                    pprint(doc)
            else:
                sm = doc.get('salary_max')
                smd = int(sm)
                if smd>salarymin:
                    pprint(doc)
        elif doc.get('salary_currency')!='':
            sm = doc.get('salary_currency')
            smd = int(sm)
            if smd>salarymin:
                pprint(doc)
            
        elif doc.get('salary_min')!='':
            sm = doc.get('salary_min')
            smd = int(sm)
            if smd>salarymin:
                pprint(doc)
        else:
            pass
        

In [121]:
salarymin = int(input('Введите значение минимума в рублях (нам нужны зарплаты более): '))

Введите значение минимума в рублях (нам нужны зарплаты более): 200000


In [120]:
vacmin(salarymin)

{'_id': 'https://hh.ru/vacancy/84255351?from=vacancy_search_list&query=Python',
 'key': 'Python',
 'salary_currency': '',
 'salary_max': '380000',
 'salary_min': '250000',
 'v': '₽',
 'vacancy': 'Data Scientist middle+/Senior'}
{'_id': 'https://hh.ru/vacancy/84001239?from=vacancy_search_list&query=Python',
 'key': 'Python',
 'salary_currency': '',
 'salary_max': '300000',
 'salary_min': '',
 'v': '₽',
 'vacancy': 'Программист Python'}
{'_id': 'https://hh.ru/vacancy/84018264?from=vacancy_search_list&query=Python',
 'key': 'Python',
 'salary_currency': '',
 'salary_max': '300000',
 'salary_min': '250000',
 'v': '₽',
 'vacancy': 'Middle+ Python Developer (backend)'}
